In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm_notebook
import time

Only want seller info but getting it through game iterations. For every game find seller and get their rating history. If seller already in database skip game. Keep going untill all games on market have seller information collected.

Becuase of cookie prompts making scraping very slow with selenium I scrape the first 100 pages for links to games. I will then use selnium on all these links on the same chrome iteration. Hopefully bipassing the cookie prompts.

In [8]:
def get_soup(url):
    '''
    gets a soup from the www.g2a.com website
    '''
    header={'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36',
           'referer':'https://www.g2a.com/en/category/games-c189?sort=newest-first&drm%5B5%5D=1',
           'authority':'www.g2a.com',
           'method':'GET',
           'path':'/en/',
           'scheme':'https',
           'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
           'accept-encoding':'gzip, deflate, br',
           'accept-language':'en-GB,en-US;q=0.9,en;q=0.8'}
    r = requests.get(url, headers=header)
    soup = BeautifulSoup(r.text)
    return soup

def find_games(soup):
    '''
    finds all games on page
    returns list
    '''
    games=soup.find('ul', class_='products-grid').findAll('li', class_='products-grid__item')
    return games

def get_game_url(game):
    '''
    from the game_card soup
    gets the url of the game
    returns full url
    '''
    url_start='https://www.g2a.com'
    url_end=game.find('h3', class_='Card__title').find('a')['href']
    return url_start+url_end

def collect_game_urls(pages):
    game_urls=[]
    for page in tqdm_notebook(range(pages)):
        if page == 0:
            url=f'https://www.g2a.com/category/games-c189?drm%5B5%5D=1'
        else:
            url=f'https://www.g2a.com/category/games-c189?drm%5B5%5D=1&page={page+1}'
        game_locals=find_games(get_soup(url))
        page_game_urls=[get_game_url(game) for game in game_locals]
        game_urls.extend(page_game_urls)
    
    return game_urls

In [10]:
game_urls=collect_game_urls(100)

KeyboardInterrupt: 

KeyboardInterrupt: 

In [16]:
def get_seller_history(game_urls, length):
    '''
    with game_url opens remote chrome
    bypasses cookie prompt
    clicks on seller history
    collects seller history soup for bs4 data collection
    returns soup
    '''
    
    options=Options()
    options.add_argument('user=agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.3 Safari/605.1.15')
    options.add_argument('referer=https://www.g2a.com/en/category/games-c189?sort=newest-first&drm%5B5%5D=1')
    options.add_argument('authority=www.g2a.com')
    options.add_argument('method=GET')
    options.add_argument('path=/en/')
    options.add_argument('scheme=https')
    options.add_argument('accept=text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3')
    options.add_argument('accept-encoding=gzip, deflate, br')
    options.add_argument('accept-language=en-GB,en-US;q=0.9,en;q=0.8')
    driver=webdriver.Chrome(options=options)
    driver.implicitly_wait(10)
    
    seller_rs=[]
    sellers=[]
    for game in game_urls:
        driver.get(game)

        #ignore cookies prompt
        #maybe_later_click(driver=driver)
        try:
            tick_loc1='/html/body/div[57]/div/div/div/div/div[2]/div[2]/div[1]/label/i'
            tick_loc2='/html/body/div[58]/div/div/div/div/div[2]/div[2]/div[1]/label/i'
            confirm_loc1='/html/body/div[57]/div/div/div/div/div[3]/button[1]'
            confirm_loc2='/html/body/div[58]/div/div/div/div/div[3]/button[1]'
            try:
                tick=driver.find_element_by_xpath(tick_loc1)
                confirm=driver.find_element_by_xpath(confirm_loc1)
            except:
                tick=driver.find_element_by_xpath(tick_loc2)
                confirm=driver.find_element_by_xpath(confirm_loc2)

            tick.click()
            confirm.click()
        except:
            print('cookie not prompted')
    
        #check if seller is already in database
        time.sleep(1)
        game_soup=BeautifulSoup(driver.page_source)
        seller = game_soup.find('span', class_='seller-info__user').text
        if seller in sellers:
            skip = True
        else:
            skip = False
            sellers.append(seller)

        #get seller history page soup
        entry_path='//*[@id="app"]/div/div[2]/div/article/header/div/div[3]/div[1]/div[1]/button/span[1]'
        seller_button=driver.find_element_by_xpath(entry_path)
        seller_button.click()


        load_more(driver, times=int(length/10))
        if skip == False:
            seller_rs.append(BeautifulSoup(driver.page_source))
        else:
            continue
    return seller_rs

def load_more(driver, times):
    button_location1='/html/body/div[5]/div/div/div/div/div[3]/div/div/div/div/button'
    button_location2='/html/body/div[6]/div/div/div/div/div[3]/div/div/div/div/button'
    for t in range(times):
        try:
            button=driver.find_element_by_xpath(button_location1)
        except:
            try:
                button=driver.find_element_by_xpath(button_location2)
            except:
                #button not there because on last page
                pass
        
        click=False
        while click==False:
            time.sleep(0.5)
            try:
                button.click()
                click=True
            except:
                continue
            
        
        
def data_collection_from_game(seller_soup):
    '''
    once the html is generated it can then be scraped
    this collection iterates over comments collecting
    the rating, date, desc and comment per buyer aswell as
    seller name and overall rating per seller. creating a single
    dictionary with keys of equal length.
    '''
    seller=seller_soup.find('div', class_='user-name').find('strong').text
    ovrall_rating=seller_soup.find('div', class_='rating-data').text
    
    comments=seller_soup.find('div', class_='comments').findAll('li')
    
    game_data={
    'seller':[],
    'ovrall_rating':[],
    'rating':[],
    'date':[],
    'desc':[],
    'comment':[]
    }
    
    for comment in comments:
        game_data['seller'].append(seller)
        game_data['ovrall_rating'].append(ovrall_rating)
        
        try:
            game_data['rating'].append(comment.find('use')['xlink:href'])
        except:
            game_data['rating'].append(np.nan)
        try:
            game_data['date'].append(comment.find('span', class_='date').text)
        except:
            game_data['date'].append(np.nan)
        try:
            game_data['desc'].append(comment.find('p', class_='comments__item-type').text)
        except:
            game_data['desc'].append(np.nan)
        try:
            game_data['comment'].append(comment.find('div', class_='comments__item-content').text)
        except:
            game_data['comment'].append(np.nan)
    
    return game_data

def seller_spider(game_urls, revs_per_seller):
    seller_rs=get_seller_history(game_urls, length=revs_per_seller)
    all_data={
    'seller':[],
    'ovrall_rating':[],
    'rating':[],
    'date':[],
    'desc':[],
    'comment':[]
    }
    for seller_r in tqdm_notebook(seller_rs):
        game_data=data_collection_from_game(seller_r)
        for key in all_data.keys():
            all_data[key].extend(game_data[key])
        tmp_data=pd.DataFrame(all_data)
        tmp_data.to_csv('backup.csv')
    
    return pd.DataFrame(all_data)

In [ ]:
data = seller_spider(game_urls, revs_per_seller=1000)

cookie not prompted
cookie not prompted
cookie not prompted


In [ ]:
data.to_csv('seller_data.csv')